## Quality control
To see whether a dataset is worth processing, it is necessary to assess the quality of the dataset.This set of code is run on the raw reads from the Illumina sequencer:

In [1]:
# /bin/sh 
# ----------------Parameters---------------------- #
#$  -S /bin/sh
#$ -q sThC.q
#$ -l mres=8G,h_data=8G,h_vmem=8G
#$ -l cpu_arch='haswell|skylake'
#$ -cwd
#$ -j y
#$ -N fastqc_unrefined_v4
#$ -o ../logs/fastqc_unrefined_v4.log
#
# ----------------Modules------------------------- #
module load bioinformatics/fastqc
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
#
fastqc ../data/working/*.fastq \
--outdir=../data/results
#
echo = `date` job $JOB_NAME done

SyntaxError: invalid syntax (<ipython-input-1-6383d7c62d08>, line 13)

This script does what it needs to, but kind of indiscriminately dumps everything in control. Kind of want to add a step where it puts everything together in a nice little box. The next step is to trim reads using Trimgalore:

In [3]:
#need to write this script

### Read filtering
To remove host reads, I have used Bowtie2's "build" function to create an index of the genome that was available, then aligned those using bowtie2. Then you separate mapped and unmapped reads using samtools, which can also separate them into different files and re-purpose them into fastq files. 

In [ ]:
#this is how you build a bowtie2 index from a known genome:
bowtie2-build ../data//bowtie-index/M.cavernosa.fasta M.cavernosa_DB

#split out: bowtie2-build path/to/input index-name

In [2]:
for f in Coral1 Coral2 Coral4 Coral5
do
bowtie2 -p 8 -x ../reference/bowtie-index/M.cavernosa -1 ../../00_raw/"$f"_R1_001_val_1.fq -2 ../../00_raw/"$f"_R2_001_val_2.fq -S ../data/working/"$f"_mapped_and_unmapped.sam
#this re-aligns your reads back to the index;

samtools view -bS ../data/working/"$f"_mapped_and_unmapped.sam > ../data/working/"$f"_mapped_and_unmapped.bam
#this converts the sam file from bowtie to a bam file for processing

samtools view -b -f 12 -F 256 ../data/working/"$f"_mapped_and_unmapped.bam > ../data/working/"$f"_bothReadsUnmapped.bam
#this extracts only the reads of which both do not match against the host genome

samtools sort -n -m 5G -@ 2 ../data/working/"$f"_bothReadsUnmapped.bam -o ../data/working/"$f"_bothReadsUnmapped_sorted.bam
samtools fastq -@ 8 ../data/working/"$f"_bothReadsUnmapped_sorted.bam \
    -1 "$f"_host_removed_R1.fastq \
    -2 "$f"_host_removed_R2.fastq \
    -0 /dev/null -s /dev/null -n

#this sorts the file so both mates are together and then extracts them back as .fastq files
# this is the most memory intensive step, is recommended to run as a .job prompt


SyntaxError: invalid syntax (<ipython-input-2-f2e05df8588e>, line 1)

This leaves you with trimmed, filtered reads. After this, you should run FastQC again. Be mindful that you change its logging location and the name of the directory, OR IT WILL OVERWRITE THE OTHER FILES